In [3]:
from torch import nn
import torch
import torchaudio
from urbansounddataset import UrbanSoundDataset
from cnn import CNNNetwork
from main_train import AUDIO_DIR,ANNOTATIONS_FILE,SAMPLE_RATE,NUM_SAMPLES

## 4번 강의 코드 실행해보기

In [33]:
import pandas as pd
annotations_file =  "UrbanSound8K/UrbanSound8K/metadata/UrbanSound8K.csv"
audio_dir = "UrbanSound8K/UrbanSound8K/audio"


SAMPLE_RATE=  22050 # 가지고 싶은 샘플 수를 결정 16000 -> 22050로 변경
NUM_SAMPLES = 22050

annotations = pd.read_csv(annotations_file)

In [39]:
annotations

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
...,...,...,...,...,...,...,...,...
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn


In [53]:
import os 
audio_dir = "UrbanSound8K/UrbanSound8K/audio/"
fold = f"fold{annotations.iloc[0,5]}"
path  = os.path.join(audio_dir,fold,annotations.iloc[0,0])
signal,sr = torchaudio.load(path)
print("path:",path)
print()
print(signal)
print()
print(sr)

path: UrbanSound8K/UrbanSound8K/audio/fold5\100032-3-0-0.wav

tensor([[-4.5776e-03, -4.8828e-03, -4.6082e-03,  ..., -5.7983e-04,
         -4.2725e-04,  3.0518e-05],
        [-4.5166e-03, -4.7913e-03, -4.6082e-03,  ..., -7.3242e-04,
         -5.4932e-04, -3.0518e-05]])

44100


## 강의 5번 실행해보기

In [59]:
audio_dir = "UrbanSound8K/UrbanSound8K/audio/"
index = 0
fold = f"fold{annotations.iloc[index,5]}"
path  = os.path.join(audio_dir,fold,annotations.iloc[index,0])
signal,sr = torchaudio.load(path)
print("signal", signal)
print()
print("size of signal", signal.size())
print()
print("sample_rate",sr)

signal tensor([[-4.5776e-03, -4.8828e-03, -4.6082e-03,  ..., -5.7983e-04,
         -4.2725e-04,  3.0518e-05],
        [-4.5166e-03, -4.7913e-03, -4.6082e-03,  ..., -7.3242e-04,
         -5.4932e-04, -3.0518e-05]])

size of signal torch.Size([2, 14004])

sample_rate 44100


In [114]:
target_sample_rate = 16000
def _resample_if_necessary(signal,sr):
# 모든 신호마다 샘플링 속도가 같은게 아니므로, 모두 같은 속도를 가질 수 있게 재샘플링을 진행해야 한다. 
    if sr != target_sample_rate:
        resampler = torchaudio.transforms.Resample(sr,target_sample_rate)
        signal = resampler(signal)
    return signal

print(f"before  signal : {signal}")
print()
after_signal = _resample_if_necessary(signal,sr)
print(f"after  signal : {after_signal}")

before  signal : tensor([[-4.5776e-03, -4.8828e-03, -4.6082e-03,  ..., -5.7983e-04,
         -4.2725e-04,  3.0518e-05],
        [-4.5166e-03, -4.7913e-03, -4.6082e-03,  ..., -7.3242e-04,
         -5.4932e-04, -3.0518e-05]])

after  signal : tensor([[-0.0031, -0.0051, -0.0048,  ..., -0.0036, -0.0025, -0.0008],
        [-0.0031, -0.0051, -0.0046,  ..., -0.0036, -0.0026, -0.0010]])


In [68]:
signal.size()

torch.Size([2, 14004])

In [70]:
def _mix_down_if_necessary( signal):
         # 만약 신호의 채널이 1개가 아니라면 신호를 믹스다운 해준다.
        if signal.shape[0] > 1: # (2,1000) 이럴 경우 
            signal = torch.mean(signal,dim=0,keepdim=True)
        return signal 

In [84]:
print(f"before size of signal : {signal.size()}")
print()
after_signal = _mix_down_if_necessary(signal)
print(f"after size of signal : {after_signal.size()}")

before size of signal : torch.Size([2, 14004])

after size of signal : torch.Size([1, 14004])


In [79]:
import numpy as np
torch.mean(signal,dim=0).shape

torch.Size([14004])

In [83]:
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
          sample_rate=SAMPLE_RATE,
          n_fft=1024,
          hop_length=512,
          n_mels=64
    )

transformation = mel_spectrogram

print(f"before signal : {signal}")
print()
after_signal = transformation(signal)
print(f"after signal : {after_signal}")

before signal : tensor([[-4.5776e-03, -4.8828e-03, -4.6082e-03,  ..., -5.7983e-04,
         -4.2725e-04,  3.0518e-05],
        [-4.5166e-03, -4.7913e-03, -4.6082e-03,  ..., -7.3242e-04,
         -5.4932e-04, -3.0518e-05]])

after signal : tensor([[[0.0054, 0.0013, 0.0026,  ..., 0.0041, 0.0022, 0.0010],
         [0.0040, 0.0050, 0.0059,  ..., 0.0074, 0.0009, 0.0048],
         [0.0019, 0.0037, 0.0215,  ..., 0.0049, 0.0019, 0.0066],
         ...,
         [0.0003, 0.0004, 0.0007,  ..., 0.0003, 0.0004, 0.0004],
         [0.0003, 0.0003, 0.0005,  ..., 0.0004, 0.0005, 0.0004],
         [0.0004, 0.0004, 0.0004,  ..., 0.0003, 0.0004, 0.0003]],

        [[0.0048, 0.0009, 0.0028,  ..., 0.0041, 0.0020, 0.0012],
         [0.0039, 0.0042, 0.0061,  ..., 0.0076, 0.0009, 0.0045],
         [0.0024, 0.0044, 0.0205,  ..., 0.0050, 0.0020, 0.0060],
         ...,
         [0.0003, 0.0004, 0.0007,  ..., 0.0003, 0.0004, 0.0003],
         [0.0003, 0.0003, 0.0005,  ..., 0.0004, 0.0005, 0.0004],
         [0.0004

In [88]:
print(f"before  signal : {signal}")
print()
after_signal = _resample_if_necessary(signal,sr)
print(f"resampling : {after_signal}")
after_signal2 = _mix_down_if_necessary(after_signal)
print(f"resampling + mix_down : {after_signal2}")
after_signal3 = transformation(after_signal2)
print(f"resampling + mix_down + melspectrogram: {after_signal3}")


before  signal : tensor([[-4.5776e-03, -4.8828e-03, -4.6082e-03,  ..., -5.7983e-04,
         -4.2725e-04,  3.0518e-05],
        [-4.5166e-03, -4.7913e-03, -4.6082e-03,  ..., -7.3242e-04,
         -5.4932e-04, -3.0518e-05]])

original sr: 44100
target sr: 16000

resampling : tensor([[-0.0032, -0.0051, -0.0048,  ..., -0.0032, -0.0020, -0.0005],
        [-0.0031, -0.0051, -0.0047,  ..., -0.0032, -0.0021, -0.0006]])
resampling + mix_down : tensor([[-0.0032, -0.0051, -0.0047,  ..., -0.0032, -0.0021, -0.0005]])
resampling + mix_down + melspectrogram: tensor([[[2.7110e-04, 5.2016e-04, 6.6397e-05, 8.3281e-05, 4.7951e-05,
          2.0758e-05, 3.4810e-05, 6.0773e-05, 3.2076e-05, 4.9419e-05],
         [1.0270e-03, 9.5022e-04, 3.3454e-04, 4.4557e-03, 2.5491e-03,
          7.3547e-04, 3.0474e-04, 3.2635e-04, 8.9394e-05, 3.1753e-04],
         [1.1775e-03, 1.0801e-03, 2.0569e-03, 4.6966e-03, 3.1390e-03,
          1.6138e-03, 1.3815e-03, 9.5162e-04, 8.4012e-04, 1.1672e-03],
         [3.2479e-03, 5.03

## 강의 6번 진행하기

In [92]:
num_samples = 12000

def _cut_if_necessary(signal):
         # signal -> Tensor => (1, ) # (number of channel , num_samples)
        if signal.shape[1] >num_samples:
            signal = signal[:,:num_samples]
        return signal 

print(f"before size of signal : {signal.size()}")
print()
after_signal = _cut_if_necessary(signal)
print(f"after size of signal : {after_signal.size()}")

before size of signal : torch.Size([2, 14004])

after size of signal : torch.Size([2, 12000])


In [90]:
signal.size()

torch.Size([2, 14004])

In [104]:
audio_dir = "UrbanSound8K/UrbanSound8K/audio/"
index = 875
fold = f"fold{annotations.iloc[index,5]}"
path  = os.path.join(audio_dir,fold,annotations.iloc[index,0])
new_signal,sr = torchaudio.load(path)
print(new_signal.size()[1])

10803


In [119]:
num_samples = 12000

def _right_pad_if_necessary(signal):
        length_signal = signal.shape[1]
        if length_signal < num_samples: # 신호에 있는 샘플 수가 예상되는 샘플 수보다 적을 때 
            # [1,1,1] -> [1,1,1,0,0]
            num_missing_samples = num_samples -  length_signal # 해당 신호가 예상하는 샘플 수가 있기 위해서 필요한 샘플의 수 
            last_dim_padding = (0,num_missing_samples)
            signal = torch.nn.functional.pad(signal,last_dim_padding) # 필요한 만큼 padding 해주기 
        return signal  

10803

In [125]:
num_samples - after_signal3.shape[1]

11936

In [127]:
after_signal3.size()

torch.Size([1, 64, 8])

In [129]:
print(f"before  signal : {new_signal}")
print(f"size of before siganl : {new_signal.size()} ")
after_signal = _resample_if_necessary(new_signal,sr)
after_signal2 = _mix_down_if_necessary(after_signal)
num_missing_samples = num_samples - after_signal2.shape[1]
last_dim_padding = (0,num_missing_samples)
after_signal = torch.nn.functional.pad(after_signal2,last_dim_padding) 

print(f"after  signal : {after_signal}")
print(f"size of after siganl : {after_signal.size()} ")

before  signal : tensor([[-0.0002, -0.0002, -0.0004,  ...,  0.0074,  0.0067,  0.0060],
        [-0.0002, -0.0002, -0.0004,  ...,  0.0074,  0.0067,  0.0060]])
size of before siganl : torch.Size([2, 10803]) 
after  signal : tensor([[-0.0002, -0.0002, -0.0002,  ...,  0.0000,  0.0000,  0.0000]])
size of after siganl : torch.Size([1, 12000]) 


In [130]:

fianl_signal = transformation(after_signal)
print(fianl_signal.size())

torch.Size([1, 64, 24])


## 강의 8번 진행하기

In [5]:
SAMPLE_RATE=  22050
NUM_SAMPLES = 22050

mel_spectrogram = torchaudio.transforms.MelSpectrogram(
          sample_rate=SAMPLE_RATE,
          n_fft=1024,
          hop_length=512,
          n_mels=64
    )

BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = .001
ANNOTATIONS_FILE =  "UrbanSound8K/UrbanSound8K/metadata/UrbanSound8K.csv"
AUDIO_DIR = "UrbanSound8K/UrbanSound8K/audio"




if torch.cuda.is_available():
        device = "cuda"
else:
        device = "cpu"

    # ms = mel_spectrogram(siganl)
usd = UrbanSoundDataset(ANNOTATIONS_FILE,
                            AUDIO_DIR,
                            mel_spectrogram, # 위에서 정의한 mel_spectrogram을 사용
                            SAMPLE_RATE,
                            NUM_SAMPLES,
                            device)

inputs, target = usd[0]
print(inputs.size())

torch.Size([1, 64, 44])


In [6]:
conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=2
            ),

            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

In [7]:
conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=2
            ),

            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

In [8]:
conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=2
            ),

            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

In [9]:
conv4 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=2
            ),

            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

In [10]:
flatten = nn.Flatten()
linear = nn.Linear(128*5*4,10)
softmax = nn.Softmax(dim=1)

In [11]:
x = torch.randn(1, 64, 44)
print(f"input data: {x.size()}")
x = conv1(x)
print(f"the output of conv1  layer: {x.size()}")
x = conv2(x)
print(f"the output of conv2 layer: {x.size()}")
x = conv3(x)
print(f"the output of conv3 layer: {x.size()}")
x = conv4(x)
print(f"the output of conv4 layer: {x.size()}")

input data: torch.Size([1, 64, 44])
the output of conv1  layer: torch.Size([16, 33, 23])
the output of conv2 layer: torch.Size([32, 17, 12])
the output of conv3 layer: torch.Size([64, 9, 7])
the output of conv4 layer: torch.Size([128, 5, 4])


In [12]:

x = x.view(-1)
print(f"the output of flatten layer: {x.size()}")
logits = linear(x)
print(f"the output of linear layer: {logits.size()}")

the output of flatten layer: torch.Size([2560])
the output of linear layer: torch.Size([10])


## 강의 10번 진행하기